In [1]:
%run ../skeleton.py
%run ../animation.py

In [2]:
skeleton = load_skeleton(r'../skeleton.dat')
animation = load_animation(r'AVRG_MOV_STD_Jog_Start.dat')

In [6]:
import k3d


plot = k3d.plot()

skeleton.load_animation(animation, 5 )

for i in range(len(skeleton._bones)):
    if skeleton._bones[i]._parentId >= 0:
        startM = skeleton.globalMatrix(i)
        endM = skeleton.globalMatrix(skeleton._bones[i]._parentId)
        mesh = k3d.line([startM[3][:3], endM[3][:3]])
        plot += mesh
        
skeleton.load_animation(animation, 15 )

for i in range(len(skeleton._bones)):
    if skeleton._bones[i]._parentId >= 0:
        startM = skeleton.globalMatrix(i)
        endM = skeleton.globalMatrix(skeleton._bones[i]._parentId)
        mesh = k3d.line([startM[3][:3], endM[3][:3]])
        plot += mesh
plot.display()

Output()